In [1]:

import numpy as np
import math

L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [2]:

print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[ 0.1338676  -1.31787852 -0.81580438  0.10849314 -1.86438204 -2.29125958
   0.20237417  1.22667764]
 [-0.12566382 -0.85288647  0.31339589 -0.76331267 -0.05254395  0.17963946
  -1.25004825  0.33494957]
 [ 0.05911044  0.07593472  0.63216622 -0.223841    0.93323609 -1.2360208
  -2.64525418  0.48687464]
 [ 0.59815955 -0.27984435 -0.91039285 -0.19008769  0.63416699  1.99340053
   0.90365102 -0.10097996]]
K
 [[ 0.77327056  0.0451663   0.6615367  -0.26743661  0.49088125  0.54616399
   0.63656231  0.88535059]
 [ 0.28640497  0.38361717  2.421019   -0.04920616  0.73570624  1.25312975
  -2.25649879 -1.55276228]
 [ 0.12004555  0.77612413 -0.90071078 -0.11322133  1.60818544 -0.44628416
   0.49328192  1.12050032]
 [-1.48390683  0.30182879 -0.26292456 -0.30120886  0.46111164 -1.08276636
   0.76887877  1.16688117]]
V
 [[ 0.47000748  1.67941975 -0.58124951  0.24262047 -0.96695464 -0.63551672
   1.1676836   1.84772634]
 [ 0.42559906 -0.44583259  0.13010665  0.25409226 -0.60025515  0.44629166
  -1.78

In [3]:
np.matmul(q, k.T)

array([[-1.47643752, -9.05191581, -0.78564606,  2.79359396],
       [-0.15109925,  2.9202144 , -1.27887269, -0.71246065],
       [-0.9425738 ,  5.93826561,  0.8150957 ,  0.13930859],
       [ 1.78432952, -1.04850156,  1.15897674, -1.96445615]])

In [4]:
q.var(), k.var(), np.matmul(q, k.T).var()


(0.9771498803132472, 0.8885096187493557, 9.333503460424996)

In [5]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.9771498803132472, 0.8885096187493557, 1.1666879325531243)

In [6]:
scaled

array([[-0.52199949, -3.20033552, -0.27776783,  0.98768462],
       [-0.05342165,  1.0324517 , -0.45214978, -0.25189288],
       [-0.33325016,  2.09949394,  0.28817985,  0.04925302],
       [ 0.63085575, -0.37070128,  0.40976016, -0.69454013]])

MASKING


>This is to ensure words don't get context from words generated in the future.

>Not required in the encoders, but required int he decoders

In [7]:
mask = np.tril(np.ones( (L, L) ))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [8]:

mask[mask == 0] = -np.infty
mask[mask == 1] = 0
     

In [9]:
mask


array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [10]:
scaled + mask


array([[-0.52199949,        -inf,        -inf,        -inf],
       [-0.05342165,  1.0324517 ,        -inf,        -inf],
       [-0.33325016,  2.09949394,  0.28817985,        -inf],
       [ 0.63085575, -0.37070128,  0.40976016, -0.69454013]])

In [11]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T
     

In [12]:
attention = softmax(scaled + mask)


In [13]:
attention


array([[1.        , 0.        , 0.        , 0.        ],
       [0.25239615, 0.74760385, 0.        , 0.        ],
       [0.07016715, 0.79921051, 0.13062234, 0.        ],
       [0.41073752, 0.1508668 , 0.32926364, 0.10913204]])

In [14]:
new_v = np.matmul(attention, v)
new_v

array([[ 0.47000748,  1.67941975, -0.58124951,  0.24262047, -0.96695464,
        -0.63551672,  1.1676836 ,  1.84772634],
       [ 0.43680757,  0.09057291, -0.0494369 ,  0.25119683, -0.69280869,
         0.17324739, -1.03848662,  0.80453154],
       [ 0.35466767, -0.19215964,  0.06536238,  0.30770488, -0.83073937,
         0.25864518, -1.35123725,  0.60041033],
       [ 0.07910038,  0.7770173 , -0.07855452,  0.54002878, -1.34565608,
        -0.3253505 ,  0.34806036,  1.03885102]])

In [15]:
v


array([[ 0.47000748,  1.67941975, -0.58124951,  0.24262047, -0.96695464,
        -0.63551672,  1.1676836 ,  1.84772634],
       [ 0.42559906, -0.44583259,  0.13010665,  0.25409226, -0.60025515,
         0.44629166, -1.7833047 ,  0.45234188],
       [-0.14128254,  0.35456689,  0.01656988,  0.6706945 , -2.16778151,
        -0.40914444, -0.06074329,  0.83633669],
       [-1.20623507,  0.34574588,  1.23796675,  1.66042951, -1.32098073,
         0.02809306,  1.44312277, -0.5836797 ]])

Function


In [16]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [17]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[ 0.1338676  -1.31787852 -0.81580438  0.10849314 -1.86438204 -2.29125958
   0.20237417  1.22667764]
 [-0.12566382 -0.85288647  0.31339589 -0.76331267 -0.05254395  0.17963946
  -1.25004825  0.33494957]
 [ 0.05911044  0.07593472  0.63216622 -0.223841    0.93323609 -1.2360208
  -2.64525418  0.48687464]
 [ 0.59815955 -0.27984435 -0.91039285 -0.19008769  0.63416699  1.99340053
   0.90365102 -0.10097996]]
K
 [[ 0.77327056  0.0451663   0.6615367  -0.26743661  0.49088125  0.54616399
   0.63656231  0.88535059]
 [ 0.28640497  0.38361717  2.421019   -0.04920616  0.73570624  1.25312975
  -2.25649879 -1.55276228]
 [ 0.12004555  0.77612413 -0.90071078 -0.11322133  1.60818544 -0.44628416
   0.49328192  1.12050032]
 [-1.48390683  0.30182879 -0.26292456 -0.30120886  0.46111164 -1.08276636
   0.76887877  1.16688117]]
V
 [[ 0.47000748  1.67941975 -0.58124951  0.24262047 -0.96695464 -0.63551672
   1.1676836   1.84772634]
 [ 0.42559906 -0.44583259  0.13010665  0.25409226 -0.60025515  0.44629166
  -1.78